# PDI - Rebalancing data for June 2022

## Setting

In [25]:
import pandas as pd
import numpy as np
import requests

In [26]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [27]:
from defillama import DefiLlama
llama = DefiLlama()

In [28]:
import plotly.express as px
import plotly.graph_objects as go

In [29]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

## Data retrieving and filtering

### Token inclusion criteria

1. **Project and token characteristics** 
    1. The project must have a listing on Defi Llama
    1. The project’s token should have been listed on CoinGecko with pricing data at least 1 year prior to the date of inclusion in the index. 
    1. The project should have a token that is native to Ethereum L1 or L2. This excludes wrapped variants, where the underlying tokens are locked on an alt-L1.
    1. The project should be a going concern, with a dedicated team actively building, supporting and maintaining the project. 
    1. No rebasing or deflationary tokens.
    1. The project must be widely considered to be building a useful protocol or product. Projects that have ponzi characteristics at the core of their offering will not be considered.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project’s protocol or product must have significant usage.

1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The token should have at least $5mm of onchain liquidity on a single pair.
    1. The token must have shown consistent DeFi liquidity on Ethereum.

1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.

### Tokens list

#### DefiLlama

In [30]:
protocols = llama.get_all_protocols()

In [31]:
top_defi_tokens = {} 
for pr in protocols[:1000]:
    if 'Ethereum' in pr['chains'] and pr['address'] and pr['address']!='-' and pr['symbol'] != '-':
        symbol = ''.join(c for c in pr['symbol'] if c.isalnum()).upper()
        top_defi_tokens[symbol] = {
            'symbol': symbol, 
            'address': pr['address'].lower()
        }

In [32]:
top_defi_tokens['XSUSHI'] = {
    'symbol': 'XSUSHI',
    'address': "0x8798249c2e607446efb7ad49ec89dd1865ff4272",
}

#### CoinGecko

##### Tokens addresses

In [33]:
coins_list = pd.DataFrame(cg.get_coins_list(include_platform=True))

In [34]:
tokens = {} 
tokens_by_addr = {}
for index, coin in coins_list.iterrows():
    if len(coin['platforms']) >= 1 and 'ethereum' in list(coin['platforms'].keys()):
        symbol = ''.join(c for c in coin['symbol'] if c.isalnum()).upper()
        address = coin['platforms']['ethereum'].lower()
        token = {
            'symbol': symbol,
            'address': address 
        }
        tokens[symbol] = token
        tokens_by_addr[address] = token

In [35]:
tokens['USDC'], tokens['WETH']

({'symbol': 'USDC', 'address': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'},
 {'symbol': 'WETH', 'address': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'})

In [36]:
# These token's addresses cant be found in the coingecko list of eth addresses
exclude_symbols = []
for symb, token in top_defi_tokens.items():
    if token['address'] not in tokens_by_addr.keys():
        print(token)
        exclude_symbols.append(token['symbol'])

{'symbol': 'MULTI', 'address': '0x9fb9a33956351cf4fa040f65a13b835a3c8764e3'}
{'symbol': 'IB', 'address': 'fantom:0x00a35fd824c717879bf370e70ac6868b95870dfb'}
{'symbol': 'COW', 'address': 'heco:0x80861a817106665bca173db6ac2ab628a738c737'}
{'symbol': 'ABR', 'address': 'celo:0x6e512bfc33be36f2666754e996ff103ad1680cc9'}
{'symbol': 'FLAG', 'address': '0x57dd84042ec9507963016596a34edad42f7e4cce '}
{'symbol': 'LYRA', 'address': 'optimism:0x50c5725949a6f0c72e6c4a641f24049a917db0cb'}
{'symbol': 'SOLV', 'address': '0x256f2d67e52fe834726d2ddcd8413654f5eb8b53'}
{'symbol': 'SGT', 'address': '0x84810bcf08744d5862b8181f12d17bfd57d3b078'}
{'symbol': 'RSR', 'address': '0x8762db106b2c2a0bccb3a80d1ed41273552616e8'}
{'symbol': 'FHM', 'address': '0xfa1fbb8ef55a4855e5688c0ee13ac3f202486286'}
{'symbol': 'GRO', 'address': '0x09e64c2b61a5f1690ee6fbed9baf5d6990f8dfd0'}
{'symbol': 'SWPR', 'address': 'arbitrum:0xde903e2712288a1da82942dddf2c20529565ac30'}
{'symbol': 'HND', 'address': 'hundredfinance.jpg'}
{'symbol

In [37]:
for symb in exclude_symbols:
    del top_defi_tokens[symb]

In [38]:
# KEEP has this address in defillama, but etherscan says this address belongs to tBTC now
for token in top_defi_tokens.values():
    if token['address'] == "0x8daebade922df735c38c80c7ebd708af50815faa":
        print(token)
del top_defi_tokens['KEEP']

{'symbol': 'KEEP', 'address': '0x8daebade922df735c38c80c7ebd708af50815faa'}


##### Market data

In [39]:
cg_tokens_data = cg.get_coins_markets(vs_currency='USD', order='market_cap_desc')
for i in range(2, 20):
    cg_tokens_data.extend(cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', page=i))
    
cg_tokens_data = pd.DataFrame.from_records(cg_tokens_data)

In [40]:
tokens_market_data = cg_tokens_data.set_index(cg_tokens_data['symbol'].str.upper())

In [41]:
tokens_market_data.head()

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
BTC,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,30127.000000,574106213483,1,6.329513e+11,2.252628e+10,30625.000,...,2.100000e+07,21000000.0,69045.00,-56.41010,2021-11-10T14:24:11.849Z,67.810000,44284.29370,2013-07-06T00:00:00.000Z,None,2022-05-23T18:51:54.288Z
ETH,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,2054.110000,248510445085,2,NaN,1.300782e+10,2084.130,...,NaN,NaN,4878.26,-57.94750,2021-11-10T14:24:19.604Z,0.432979,473694.63795,2015-10-20T00:00:00.000Z,"{'times': 90.19474537631922, 'currency': 'btc'...",2022-05-23T18:51:15.085Z
USDT,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,0.999832,73367167187,3,NaN,4.316793e+10,1.004,...,7.327509e+10,NaN,1.32,-24.38380,2018-07-24T00:00:00.000Z,0.572521,74.74894,2015-03-02T00:00:00.000Z,None,2022-05-23T18:52:06.458Z
BNB,binancecoin,bnb,BNB,https://assets.coingecko.com/coins/images/825/...,333.710000,56149048346,4,5.614905e+10,1.949633e+09,336.210,...,1.681370e+08,168137035.9,686.31,-51.41510,2021-05-10T07:24:17.097Z,0.039818,837320.03871,2017-10-19T00:00:00.000Z,None,2022-05-23T18:51:47.148Z
USDC,usd-coin,usdc,USD Coin,https://assets.coingecko.com/coins/images/6319...,1.002000,53302637069,5,NaN,4.784784e+09,1.005,...,5.320222e+10,NaN,1.17,-14.63965,2019-05-08T00:40:28.300Z,0.891848,12.24178,2021-05-19T13:14:05.611Z,None,2022-05-23T18:52:59.372Z


### Liquidity check

In [42]:
dex_subgraph_urls = {
    'univ2': "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2",
    'sushi': "https://api.thegraph.com/subgraphs/name/zippoxer/sushiswap-subgraph-fork",
} 

In [43]:
base_tokens = [tokens['USDC'], tokens['WETH']]
base_tokens

[{'symbol': 'USDC', 'address': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'},
 {'symbol': 'WETH', 'address': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'}]

In [44]:
query = """
    query pairs ($token0_ids: [ID!]!, $token1_ids: [ID!]!) {
        pairs (first:1000, where: {token0_in: $token0_ids, token1_in: $token1_ids}){
            id
            token0 {
              id
            }
            token1 {
              id
            }
            reserveUSD
        }
    }
    """

In [45]:
base_tokens_ids = [token['address'] for token in base_tokens]
tokens_ids = [token['address'] for symb, token in top_defi_tokens.items()]

In [46]:
rows = {token['symbol']: {} for token in top_defi_tokens.values()}
for dex_name, url in dex_subgraph_urls.items():
    request = requests.post(url, json={'query': query, 'variables': {'token0_ids': base_tokens_ids, 'token1_ids': tokens_ids}})
    pairs1 = request.json()['data']['pairs']

    request = requests.post(url, json={'query': query, 'variables': {'token0_ids': tokens_ids, 'token1_ids': base_tokens_ids}})
    pairs2 = request.json()['data']['pairs']
    
    pairs = pairs1 + pairs2
    
    print(f"{dex_name}: {len(pairs)} pairs")
    
    for pair in pairs:
        if pair['token0']['id'] in base_tokens_ids:
            base_token = tokens_by_addr[pair['token0']['id']]
            token = tokens_by_addr[pair['token1']['id']]
        else:
            base_token = tokens_by_addr[pair['token1']['id']]
            token = tokens_by_addr[pair['token0']['id']]
        
        colname = f"{dex_name}_{base_token['symbol']}" 
        
        rows[token['symbol']][colname] = float(pair['reserveUSD'])

univ2: 272 pairs
sushi: 150 pairs


In [47]:
liquidities = pd.DataFrame.from_dict(rows, orient='index')
liq_check = (liquidities > 5e6).any(axis=1).sort_values(ascending=False)
liq_check_passed = liq_check[liq_check]

In [48]:
lq_passed = tokens_market_data[
    tokens_market_data.index.isin(liq_check_passed.index)
]

In [49]:
lq_passed

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
WBTC,wrapped-bitcoin,wbtc,Wrapped Bitcoin,https://assets.coingecko.com/coins/images/7598...,30127.000000,8333656611,14,8.333657e+09,2.323919e+08,30625.000000,...,2.765030e+05,2.765030e+05,70643.000000,-57.42587,2021-11-10T14:40:19.650Z,3139.170000,858.08070,2019-04-02T00:00:00.000Z,None,2022-05-23T18:52:17.956Z
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,5.840000,2666307486,33,5.840894e+09,1.618613e+08,5.950000,...,1.000000e+09,1.000000e+09,44.920000,-87.00969,2021-05-03T05:25:04.822Z,1.030000,466.41026,2020-09-17T01:20:38.214Z,None,2022-05-23T18:52:29.151Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,105.890000,1454995112,49,1.694174e+09,3.070402e+08,107.610000,...,1.600000e+07,1.600000e+07,661.690000,-84.03700,2021-05-18T21:19:59.514Z,26.020000,305.89101,2020-11-05T09:20:11.928Z,None,2022-05-23T18:52:04.619Z
COMP,compound-governance-token,comp,Compound,https://assets.coingecko.com/coins/images/1077...,73.340000,502787051,111,7.333442e+08,5.429574e+07,75.100000,...,1.000000e+07,1.000000e+07,910.540000,-91.95687,2021-05-12T02:29:08.794Z,54.910000,33.36759,2022-05-12T07:18:59.691Z,None,2022-05-23T18:53:17.277Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,1.270000,474876750,113,1.269795e+09,6.533476e+06,1.340000,...,1.000000e+09,1.000000e+09,7.300000,-82.65880,2021-08-20T08:35:20.158Z,0.505506,150.52253,2021-03-27T09:10:29.155Z,None,2022-05-23T18:51:57.177Z
FXS,frax-share,fxs,Frax Share,https://assets.coingecko.com/coins/images/1342...,7.570000,456400198,116,7.535787e+08,4.410136e+07,8.230000,...,9.949117e+07,9.949117e+07,42.800000,-82.21373,2022-01-12T15:22:27.465Z,1.500000,406.43189,2021-06-25T16:50:51.447Z,None,2022-05-23T18:52:44.131Z
YFI,yearn-finance,yfi,yearn.finance,https://assets.coingecko.com/coins/images/1184...,9646.560000,342656226,139,3.537006e+08,5.418489e+07,9790.590000,...,3.666600e+04,3.666600e+04,90787.000000,-89.37863,2021-05-12T00:29:37.713Z,31.650000,30371.70681,2020-07-18T12:26:27.150Z,None,2022-05-23T18:52:01.771Z
SUSHI,sushi,sushi,Sushi,https://assets.coingecko.com/coins/images/1227...,1.670000,321143650,152,4.164439e+08,2.008995e+08,1.690000,...,2.422526e+08,2.500000e+08,23.380000,-92.90762,2021-03-13T23:44:36.774Z,0.475381,248.76142,2020-11-04T14:53:53.560Z,None,2022-05-23T18:52:25.734Z
SYN,synapse-2,syn,Synapse,https://assets.coingecko.com/coins/images/1802...,1.300000,239778901,177,3.251096e+08,1.148520e+07,1.450000,...,NaN,2.500000e+08,4.920000,-73.53843,2021-10-24T00:24:41.805Z,0.389600,233.85430,2021-09-21T21:19:58.123Z,None,2022-05-23T18:52:42.676Z


### Non quantative checks

In [50]:
exclude_list = [
    {
        'symbol': 'WBTC',
        'reason': 'Criteria 1.C'
    },
    {
        'symbol': 'XSUSHI',
        'reason': 'Duplicates SUSHI'
    },
    {
        'symbol': 'ERN',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'NFTX',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'FXS',
        'reason': 'Criteria 1.G'
    },
    {
        'symbol': 'RGT',
        'reason': 'Deprecated asset'
    },
    {
        'symbol': 'SUPER',
        'reason': 'Not DeFi'
    }
]
exclude_symbols = [token['symbol'] for token in exclude_list]

In [51]:
nq_passed = lq_passed[
    ~lq_passed.index.isin(exclude_symbols)
]

In [52]:
nq_passed

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,5.840000,2666307486,33,5.840894e+09,1.618613e+08,5.950000,...,1.000000e+09,1.000000e+09,44.920000,-87.00969,2021-05-03T05:25:04.822Z,1.030000,466.41026,2020-09-17T01:20:38.214Z,None,2022-05-23T18:52:29.151Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,105.890000,1454995112,49,1.694174e+09,3.070402e+08,107.610000,...,1.600000e+07,1.600000e+07,661.690000,-84.03700,2021-05-18T21:19:59.514Z,26.020000,305.89101,2020-11-05T09:20:11.928Z,None,2022-05-23T18:52:04.619Z
COMP,compound-governance-token,comp,Compound,https://assets.coingecko.com/coins/images/1077...,73.340000,502787051,111,7.333442e+08,5.429574e+07,75.100000,...,1.000000e+07,1.000000e+07,910.540000,-91.95687,2021-05-12T02:29:08.794Z,54.910000,33.36759,2022-05-12T07:18:59.691Z,None,2022-05-23T18:53:17.277Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,1.270000,474876750,113,1.269795e+09,6.533476e+06,1.340000,...,1.000000e+09,1.000000e+09,7.300000,-82.65880,2021-08-20T08:35:20.158Z,0.505506,150.52253,2021-03-27T09:10:29.155Z,None,2022-05-23T18:51:57.177Z
YFI,yearn-finance,yfi,yearn.finance,https://assets.coingecko.com/coins/images/1184...,9646.560000,342656226,139,3.537006e+08,5.418489e+07,9790.590000,...,3.666600e+04,3.666600e+04,90787.000000,-89.37863,2021-05-12T00:29:37.713Z,31.650000,30371.70681,2020-07-18T12:26:27.150Z,None,2022-05-23T18:52:01.771Z
SUSHI,sushi,sushi,Sushi,https://assets.coingecko.com/coins/images/1227...,1.670000,321143650,152,4.164439e+08,2.008995e+08,1.690000,...,2.422526e+08,2.500000e+08,23.380000,-92.90762,2021-03-13T23:44:36.774Z,0.475381,248.76142,2020-11-04T14:53:53.560Z,None,2022-05-23T18:52:25.734Z
SYN,synapse-2,syn,Synapse,https://assets.coingecko.com/coins/images/1802...,1.300000,239778901,177,3.251096e+08,1.148520e+07,1.450000,...,NaN,2.500000e+08,4.920000,-73.53843,2021-10-24T00:24:41.805Z,0.389600,233.85430,2021-09-21T21:19:58.123Z,None,2022-05-23T18:52:42.676Z
TOKE,tokemak,toke,Tokemak,https://assets.coingecko.com/coins/images/1749...,4.540000,66723640,384,4.532449e+08,9.233760e+05,4.810000,...,1.000000e+08,1.000000e+08,79.020000,-94.25812,2021-11-04T00:35:29.799Z,4.480000,1.36111,2022-05-22T06:27:16.551Z,None,2022-05-23T18:51:50.291Z
ALCX,alchemix,alcx,Alchemix,https://assets.coingecko.com/coins/images/1411...,34.440000,44752672,492,8.198494e+07,1.648500e+06,34.730000,...,1.998377e+06,2.393060e+06,2066.200000,-98.33649,2021-03-20T19:33:19.740Z,28.550000,20.38538,2022-05-12T22:34:00.644Z,None,2022-05-23T18:52:41.991Z


### Retrieve historical data & livetime check

In [53]:
exclude_list = []

prices_data = pd.DataFrame()
marketcaps = pd.DataFrame() 
for index, data in nq_passed[['id', 'symbol']].iterrows():
    id_ = data['id']
    symbol = data['symbol'].upper()
    data = cg.get_coin_market_chart_by_id(id_, vs_currency='USD', days='max')
  
    df_prices = pd.DataFrame(data['prices'], columns=['date', symbol])
    df_prices = df_prices[df_prices[symbol] > 0]
    df_prices['date'] = pd.to_datetime(df_prices['date'], unit='ms').dt.date
    df_prices['date'] = pd.to_datetime(df_prices['date'])
    df_prices = df_prices.set_index('date', drop=True)
    df_prices = df_prices.loc[~df_prices.index.duplicated(keep='first')]
    
    if len(df_prices) < 365:
        print(f'Excluding {symbol}, prices data available only for {len(df_prices)} < 365 days')
        exclude_list.append(symbol)
        continue
    prices_data = pd.concat([prices_data, df_prices], axis=1)
    
    df_mcaps = pd.DataFrame(data['market_caps'], columns=['date', symbol])
    df_mcaps = df_mcaps[df_mcaps[symbol] > 0]
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'], unit='ms').dt.date
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'])
    df_mcaps = df_mcaps.set_index('date', drop=True)
    df_mcaps = df_mcaps.loc[~df_mcaps.index.duplicated(keep='first')]
    
    if len(df_mcaps) < 365:
        print(f'Note: {symbol}, marketcap data available only for {len(df_mcaps)} < 365 days')
        ## continue
    df_mcaps = df_mcaps.reindex(df_prices.index)
    marketcaps = pd.concat([marketcaps, df_mcaps], axis=1)
    
    

Excluding SYN, prices data available only for 267 < 365 days
Excluding TOKE, prices data available only for 290 < 365 days
Excluding FLX, prices data available only for 162 < 365 days
Excluding MIST, prices data available only for 166 < 365 days


In [54]:
lt_passed = nq_passed[
    ~nq_passed.index.isin(exclude_list)
]

In [55]:
lt_passed

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,5.84,2666307486,33,5.840894e+09,161861308.0,5.95,...,1.000000e+09,1.000000e+09,44.92,-87.00969,2021-05-03T05:25:04.822Z,1.030000,466.41026,2020-09-17T01:20:38.214Z,None,2022-05-23T18:52:29.151Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,105.89,1454995112,49,1.694174e+09,307040178.0,107.61,...,1.600000e+07,1.600000e+07,661.69,-84.03700,2021-05-18T21:19:59.514Z,26.020000,305.89101,2020-11-05T09:20:11.928Z,None,2022-05-23T18:52:04.619Z
COMP,compound-governance-token,comp,Compound,https://assets.coingecko.com/coins/images/1077...,73.34,502787051,111,7.333442e+08,54295744.0,75.10,...,1.000000e+07,1.000000e+07,910.54,-91.95687,2021-05-12T02:29:08.794Z,54.910000,33.36759,2022-05-12T07:18:59.691Z,None,2022-05-23T18:53:17.277Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,1.27,474876750,113,1.269795e+09,6533476.0,1.34,...,1.000000e+09,1.000000e+09,7.30,-82.65880,2021-08-20T08:35:20.158Z,0.505506,150.52253,2021-03-27T09:10:29.155Z,None,2022-05-23T18:51:57.177Z
YFI,yearn-finance,yfi,yearn.finance,https://assets.coingecko.com/coins/images/1184...,9646.56,342656226,139,3.537006e+08,54184887.0,9790.59,...,3.666600e+04,3.666600e+04,90787.00,-89.37863,2021-05-12T00:29:37.713Z,31.650000,30371.70681,2020-07-18T12:26:27.150Z,None,2022-05-23T18:52:01.771Z
SUSHI,sushi,sushi,Sushi,https://assets.coingecko.com/coins/images/1227...,1.67,321143650,152,4.164439e+08,200899472.0,1.69,...,2.422526e+08,2.500000e+08,23.38,-92.90762,2021-03-13T23:44:36.774Z,0.475381,248.76142,2020-11-04T14:53:53.560Z,None,2022-05-23T18:52:25.734Z
ALCX,alchemix,alcx,Alchemix,https://assets.coingecko.com/coins/images/1411...,34.44,44752672,492,8.198494e+07,1648500.0,34.73,...,1.998377e+06,2.393060e+06,2066.20,-98.33649,2021-03-20T19:33:19.740Z,28.550000,20.38538,2022-05-12T22:34:00.644Z,None,2022-05-23T18:52:41.991Z
BOND,barnbridge,bond,BarnBridge,https://assets.coingecko.com/coins/images/1281...,3.66,24649567,660,3.660414e+07,14368470.0,3.68,...,1.000000e+07,1.000000e+07,185.69,-98.03838,2020-10-27T00:36:32.813Z,3.060000,18.91411,2022-05-14T16:14:28.237Z,None,2022-05-23T18:51:31.253Z


### Marketcap ranking & top10 filtering

In [56]:
chosen_tokens = lt_passed.sort_values("market_cap", ascending=False).iloc[:10]

In [57]:
chosen_tokens

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,5.84,2666307486,33,5.840894e+09,161861308.0,5.95,...,1.000000e+09,1.000000e+09,44.92,-87.00969,2021-05-03T05:25:04.822Z,1.030000,466.41026,2020-09-17T01:20:38.214Z,None,2022-05-23T18:52:29.151Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,105.89,1454995112,49,1.694174e+09,307040178.0,107.61,...,1.600000e+07,1.600000e+07,661.69,-84.03700,2021-05-18T21:19:59.514Z,26.020000,305.89101,2020-11-05T09:20:11.928Z,None,2022-05-23T18:52:04.619Z
COMP,compound-governance-token,comp,Compound,https://assets.coingecko.com/coins/images/1077...,73.34,502787051,111,7.333442e+08,54295744.0,75.10,...,1.000000e+07,1.000000e+07,910.54,-91.95687,2021-05-12T02:29:08.794Z,54.910000,33.36759,2022-05-12T07:18:59.691Z,None,2022-05-23T18:53:17.277Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,1.27,474876750,113,1.269795e+09,6533476.0,1.34,...,1.000000e+09,1.000000e+09,7.30,-82.65880,2021-08-20T08:35:20.158Z,0.505506,150.52253,2021-03-27T09:10:29.155Z,None,2022-05-23T18:51:57.177Z
YFI,yearn-finance,yfi,yearn.finance,https://assets.coingecko.com/coins/images/1184...,9646.56,342656226,139,3.537006e+08,54184887.0,9790.59,...,3.666600e+04,3.666600e+04,90787.00,-89.37863,2021-05-12T00:29:37.713Z,31.650000,30371.70681,2020-07-18T12:26:27.150Z,None,2022-05-23T18:52:01.771Z
SUSHI,sushi,sushi,Sushi,https://assets.coingecko.com/coins/images/1227...,1.67,321143650,152,4.164439e+08,200899472.0,1.69,...,2.422526e+08,2.500000e+08,23.38,-92.90762,2021-03-13T23:44:36.774Z,0.475381,248.76142,2020-11-04T14:53:53.560Z,None,2022-05-23T18:52:25.734Z
ALCX,alchemix,alcx,Alchemix,https://assets.coingecko.com/coins/images/1411...,34.44,44752672,492,8.198494e+07,1648500.0,34.73,...,1.998377e+06,2.393060e+06,2066.20,-98.33649,2021-03-20T19:33:19.740Z,28.550000,20.38538,2022-05-12T22:34:00.644Z,None,2022-05-23T18:52:41.991Z
BOND,barnbridge,bond,BarnBridge,https://assets.coingecko.com/coins/images/1281...,3.66,24649567,660,3.660414e+07,14368470.0,3.68,...,1.000000e+07,1.000000e+07,185.69,-98.03838,2020-10-27T00:36:32.813Z,3.060000,18.91411,2022-05-14T16:14:28.237Z,None,2022-05-23T18:51:31.253Z


In [58]:
chosen_tokens_symbols = list(chosen_tokens.index)
prices_data = prices_data[chosen_tokens_symbols]
marketcaps = marketcaps[chosen_tokens_symbols]

In [59]:
without_nan_index = (marketcaps.isnull().sum(axis=1) == 0) & (prices_data.isnull().sum(axis=1) == 0)
marketcaps = marketcaps[without_nan_index]
prices_data = prices_data[without_nan_index]

In [60]:
prices_data

,UNI,AAVE,COMP,LDO,YFI,SUSHI,ALCX,BOND
date,,,,,,,,
2021-03-02,25.592699,386.507922,494.358379,0.849448,34900.371111,18.798040,1050.297397,56.050115
2021-03-03,25.673868,392.581993,501.438114,0.879645,33400.696195,17.925825,991.795750,52.260027
2021-03-04,25.486634,397.858343,494.206140,0.836782,33881.025287,17.517664,1376.135328,57.280345
2021-03-05,28.368215,376.610205,472.457882,0.687990,32560.148658,17.408828,1139.907512,55.329831
2021-03-06,28.308459,369.647205,479.153858,0.783612,32137.453888,16.690631,1167.196998,58.316153
...,...,...,...,...,...,...,...,...
2022-05-19,4.889555,81.728519,64.481027,1.370136,9083.386841,1.259283,30.955426,3.392257
2022-05-20,5.196583,91.479758,67.672880,1.420394,9541.809737,1.370140,32.999642,3.598067
2022-05-21,5.113348,90.366300,66.126189,1.319123,9200.355707,1.296737,31.688564,3.426192


In [61]:
marketcaps

,UNI,AAVE,COMP,LDO,YFI,SUSHI,ALCX,BOND
date,,,,,,,,
2021-03-02,7.871645e+09,4.751222e+09,2.142571e+09,1.181185e+07,1.237769e+09,2.362758e+09,3.338203e+07,7.552599e+07
2021-03-03,7.938771e+09,4.878411e+09,2.175709e+09,1.224257e+07,1.188119e+09,2.273568e+09,3.126962e+07,7.110369e+07
2021-03-04,7.933740e+09,4.948727e+09,2.154215e+09,1.185368e+07,1.210977e+09,2.240011e+09,4.418676e+07,7.802084e+07
2021-03-05,8.803390e+09,4.684718e+09,2.054969e+09,9.733795e+06,1.156134e+09,2.211941e+09,3.647704e+07,7.517484e+07
2021-03-06,8.703822e+09,4.580840e+09,2.077348e+09,1.101787e+07,1.153659e+09,2.123790e+09,3.716741e+07,7.902313e+07
...,...,...,...,...,...,...,...,...
2022-05-19,2.233139e+09,1.128029e+09,4.427092e+08,4.964952e+08,3.237477e+08,2.443990e+08,4.030301e+07,2.284517e+07
2022-05-20,2.372593e+09,1.256450e+09,4.639994e+08,5.174210e+08,3.389635e+08,2.640117e+08,4.293617e+07,2.415870e+07
2022-05-21,2.334569e+09,1.242585e+09,4.529641e+08,4.867826e+08,3.272163e+08,2.504307e+08,4.131932e+07,2.301672e+07


## Weighting 

**Weighting requirements**
* The maximum weight any one token can have is 30%.
* All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
* Any asset with a weight below 0.5% will be removed from the index.


### Weight mcaps

In [62]:
weights = marketcaps.div(marketcaps.sum(axis=1), axis=0)
weights = weights.sort_values(weights.last_valid_index(), axis=1, ascending=False)

In [63]:
fig = px.line(weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    ## showlegend=False,
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights without max constraint'
)

### Adjust weights

This process adjusts weights to adhere to the weight constraints described in our methodology. In addition the `remove_tiny_weights` function checks for any assets with a weight below the minimum threshold of $0.5\%$ and proportionally increases all remaining weights (which are $> 0.5\%$). Note that after this procedure readjusting weights to respect maximum may be needed.

In [64]:
def adjust_weights(weights, max_weight):
    w = weights.copy()
    
    while (w > max_weight).any(axis=None):
        w[w > max_weight] = max_weight
        c = 1 - w.sum(axis=1)
        w_less = w[w < max_weight]
        w[(w < max_weight) & (w > 0)] += w_less.div(w_less.sum(axis=1), axis=0).mul(c, axis=0)
    
    return w

In [65]:
def remove_tiny_weights(weights, min_weight):
    w = weights.copy()
    
    w[w < min_weight] = 0
    w = w.div(w.sum(axis=1), axis=0)
    
    return w

In [66]:
adjusted_weights = adjust_weights(weights, max_weight=0.3) 
adjusted_weights = remove_tiny_weights(adjusted_weights, min_weight=0.005)
adjusted_weights = adjust_weights(adjusted_weights, max_weight=0.3)

adjusted_weights = adjusted_weights.sort_values(adjusted_weights.last_valid_index(),ascending=False, axis = 1)
adjusted_weights

,UNI,AAVE,LDO,COMP,YFI,SUSHI,ALCX,BOND
date,,,,,,,,
2021-03-02,0.3,0.300000,0.000000,0.147291,0.085090,0.162427,0.000000,0.005192
2021-03-03,0.3,0.300000,0.000000,0.154377,0.084303,0.161320,0.000000,0.000000
2021-03-04,0.3,0.300000,0.000000,0.151619,0.085232,0.157658,0.000000,0.005491
2021-03-05,0.3,0.300000,0.000000,0.149501,0.084110,0.160921,0.000000,0.005469
2021-03-06,0.3,0.300000,0.000000,0.152920,0.084924,0.156339,0.000000,0.005817
...,...,...,...,...,...,...,...,...
2022-05-19,0.3,0.292611,0.128791,0.114839,0.083980,0.063397,0.010455,0.005926
2022-05-20,0.3,0.300000,0.125322,0.112383,0.082099,0.063945,0.010399,0.005851
2022-05-21,0.3,0.300000,0.123101,0.114549,0.082749,0.063331,0.010449,0.005821


In [67]:
fig = px.line(adjusted_weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights with max constraint'
)

### Convert weights to the [1, 255] scale

On the contract side weights are integer numbers from the interval $[1, 255]$ with total sum $255$, so it's needed to convert retrieved weights to this format.

Note that one can't just round weights to integers after scaling, since it doesn't guarantee that their sum will be $255$. To fix that firstly floor function is applied to the weights and then $1$ is being added to the $k$ weights with largest fractional parts.

In [109]:
def convert_weights(weights):
    w_scaled = weights * 255
    w_res = np.floor(w_scaled).astype(int)    
    remainders = w_scaled - w_res    
    k = round(remainders.sum())
    w_below_max = w_res[w_res<76]
    for i in range(k):
        w_below_max[i] +=1
    for i in w_below_max.index:
        w_res[i] = w_below_max[i]
    return w_res

In [72]:
last_weights = adjusted_weights.iloc[-1].dropna()
last_weights

UNI      0.300000
AAVE     0.300000
LDO      0.118950
COMP     0.117010
YFI      0.080785
SUSHI    0.067269
ALCX     0.010269
BOND     0.005717
Name: 2022-05-23 00:00:00, dtype: float64

In [112]:
converted_last_weights = convert_weights(last_weights)
converted_last_weights

UNI      76
AAVE     76
LDO      31
COMP     30
YFI      21
SUSHI    18
ALCX      2
BOND      1
Name: 2022-05-23 00:00:00, dtype: int64

## Summary table

In [115]:
pdi = pd.DataFrame()
pdi.index = chosen_tokens.index
pdi['name'] = chosen_tokens['name']
pdi['market_cap'] = marketcaps.iloc[-1].astype(int)
pdi['price'] = prices_data.iloc[-1]
pdi['weight'] = last_weights
pdi['weight_converted'] = converted_last_weights
pdi['address'] = [top_defi_tokens[symb]['address'] for symb in pdi.index]

pdi = pdi[pdi['weight'] > 0]
pdi = pdi.sort_values("market_cap",ascending=False)
pdi

,name,market_cap,price,weight,weight_converted,address
symbol,,,,,,
UNI,Uniswap,2521147054,5.512928,0.300000,76,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984
AAVE,Aave,1299371385,94.595402,0.300000,76,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9
LDO,Lido DAO,496175840,1.323604,0.118950,31,0x5a98fcbea516cf06857215779fd812ca3bef1b32
COMP,Compound,488084560,71.061189,0.117010,30,0xc00e94cb662c3520282e6f5717214004a7f26888
YFI,yearn.finance,336978469,9506.874677,0.080785,21,0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e
SUSHI,Sushi,280599895,1.455847,0.067269,18,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2
ALCX,Alchemix,42833508,32.974543,0.010269,2,0xdbdb4d16eda451d0503b854cf79d55697f90c8df
BOND,BarnBridge,23848712,3.554770,0.005717,1,0x0391d2021f89dc339f60fff84546ea23e337750f
